# Prototype Development

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

from utils import *
from plots import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Data we'll need:

In [6]:
data_path = Path("..") / "data"
counts_path = data_path / "Pedestrian_Counting_System___2009_to_Present__counts_per_hour_.csv"
sensor_path = data_path / "Pedestrian_Counting_System_-_Sensor_Locations.csv"

## Prototype

In [47]:
from pedestrian import PedestrianDataset

pedestrian = PedestrianDataset.load(counts_path, lat_long_path=sensor_path)

In [48]:
pedestrian.filter(year=2020).filter()

In [ ]:
# TODO
# -- get plots in the PedestrianDataset working; then use to make dashboard
# -- diagnose and fix issues with souns count/ sensor time series plots when one/few plots displayed
# -- add titles with current filter values to each plot
# -- sort out margins
# -- address artefcts in plots
# -- add prev 4 week average to time series plots etc
# -- see if any options for caching

from prototype import prototype

prototype(foot_df)

## Comparing Sensors by Year

In [ ]:
import functools

# inputs
sensor_input = widgets.Dropdown(options=SENSORS)
year_input = widgets.Dropdown(options=YEARS)
month_input = widgets.Dropdown(options=MONTHS)

# binding inputs to output function
out = widgets.interactive_output(
    display_output(functools.partial(plot_years, foot_df)),
    {"sensor": sensor_input, "month": month_input},
)

# layout
widgets.HBox(
    [
        widgets.VBox(
            [
                widgets.HTML("<H1>Comparing Sensors by Year</h1>"),
                sensor_input,
                month_input,
            ]
        ),
        out,
    ]
)

Ideas:
* align to eg first Monday of the month
* get rid of the 2000 from the initial x value see `tickformatstops` example near bottom of [this page](https://plotly.com/python/time-series/)
* extend to `sensor_plot`
* add ability to overlay mean/median lines from custom date (ranges)
* work out how to combine `plot_sensors` and `plot_year` into single `plot_comparison` that also allows selecting month, and even day?
* improve hover label

## Ipywidgets

* input controls
* display outputs
* binding inputs and outputs to interactions

In [7]:
import ipywidgets as widgets

In [8]:
widgets.Dropdown(options=["cat", "dog"])

Dropdown(options=('cat', 'dog'), value='cat')

In [9]:
widgets.Dropdown(options=[1,2,3,4, 5])

Dropdown(options=(1, 2, 3, 4, 5), value=1)

In [10]:
widgets.IntRangeSlider(
    value=[2009, 2020],
    min=2009,
    max=2020,
)

IntRangeSlider(value=(2009, 2020), max=2020, min=2009)